<a href="https://colab.research.google.com/github/Jaidon-Smith/AI-Karaoke/blob/main/Japanese%20STT%20Version%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploration of Tensorflow CTC Loss

# Sentence piece for grapheme based BPE https://github.com/google/sentencepiece

Upon reading the github docs, there appears to be some tensorflow integration if you search 'Sentencepiece'

# Exploring pretained tokenisations

In [ ]:
#@title Install dependencies
!pip install --quiet tensorflow-text

In [ ]:
#@title Import dependencies

In [ ]:
#@title Original hub code
!pip install tensorflow-text

import numpy as np
import tensorflow.compat.v1 as tf
import tensorflow_hub as hub
import tensorflow_text as tf_text
tf.disable_eager_execution()

n_layer = 12
d_model = 768
max_gen_len = 128

def generate(module, inputs, mems):
  """Generate text."""
  inputs = tf.dtypes.cast(inputs, tf.int64)
  generation_input_dict = dict(input_tokens=inputs)
  mems_dict = {}
  for i in range(n_layer):
    mems_dict["mem_{}".format(i)] = mems[i]
  generation_input_dict.update(mems_dict)

  generation_outputs = module(generation_input_dict, signature="prediction",
                              as_dict=True)
  probs = generation_outputs["probs"]

  new_mems = []
  for i in range(n_layer):
    new_mems.append(generation_outputs["new_mem_{}".format(i)])

  return probs, new_mems

g = tf.Graph()
with g.as_default():
  module = hub.Module("https://tfhub.dev/google/wiki40b-lm-ja/1")
  text = ["\n_START_ARTICLE_\nしのぶ・まさみshow'05 恋してラララ\n_START_SECTION_\n概要\n_START_PARAGRAPH_\n『上海ルーキーSHOW』の打ち切り後に放送された年末特番で、同番組MCの大竹しのぶと久本雅美が恋愛にまつわるテーマでトークや音楽企画を展開していた。基本は女"]

  # Word embeddings.
  embeddings = module(dict(text=text), signature="word_embeddings",
                      as_dict=True)
  embeddings = embeddings["word_embeddings"]

  # Activations at each layer.
  activations = module(dict(text=text),signature="activations", as_dict=True)
  activations = activations["activations"]

  # Negative log likelihood of the text, and perplexity.
  neg_log_likelihood = module(dict(text=text), signature="neg_log_likelihood",
                              as_dict=True)
  neg_log_likelihood = neg_log_likelihood["neg_log_likelihood"]
  ppl = tf.exp(tf.reduce_mean(neg_log_likelihood, axis=1))

  # Tokenization and detokenization with the sentencepiece model.
  token_ids = module(dict(text=text), signature="tokenization", as_dict=True)
  token_ids = token_ids["token_ids"]

  detoken_text = module(dict(token_ids=token_ids), signature="detokenization",
                        as_dict=True)
  detoken_text = detoken_text["text"]

  # Generation
  mems_np = [np.zeros([1, 0, d_model], dtype=np.float32) for _ in range(n_layer)]
  inputs_np = token_ids
  sampled_ids = []
  for step in range(max_gen_len):
    probs, mems_np = generate(module, inputs_np, mems_np)
    sampled_id = tf.random.categorical(tf.math.log(probs[0]), num_samples=1, dtype=tf.int32)
    sampled_id = tf.squeeze(sampled_id)

    sampled_ids.append(sampled_id)
    inputs_np = tf.reshape(sampled_id, [1, 1])

  sampled_ids = tf.expand_dims(sampled_ids, axis=0)
  generated_text = module(dict(token_ids=sampled_ids),
                          signature="detokenization", as_dict=True)
  generated_text = generated_text["text"]

  init_op = tf.group([tf.global_variables_initializer(),
                      tf.tables_initializer()])

# Initialize session.
with tf.Session(graph=g) as session:
  session.run(init_op)
  embeddings, neg_log_likelihood, ppl, activations, token_ids, detoken_text, generated_text = session.run([
    embeddings, neg_log_likelihood, ppl, activations, token_ids, detoken_text, generated_text])

In [ ]:
#@title Original hub code without generation
!pip install tensorflow-text

import numpy as np
import tensorflow.compat.v1 as tf
import tensorflow_hub as hub
import tensorflow_text as tf_text
tf.disable_eager_execution()

n_layer = 12
d_model = 768
max_gen_len = 128


g = tf.Graph()
with g.as_default():
  module = hub.Module("https://tfhub.dev/google/wiki40b-lm-ja/1")
  text = ["\n_START_ARTICLE_\nしのぶ・まさみshow'05 恋してラララ\n_START_SECTION_\n概要\n_START_PARAGRAPH_\n『上海ルーキーSHOW』の打ち切り後に放送された年末特番で、同番組MCの大竹しのぶと久本雅美が恋愛にまつわるテーマでトークや音楽企画を展開していた。基本は女"]

  # Word embeddings.
  embeddings = module(dict(text=text), signature="word_embeddings",
                      as_dict=True)
  embeddings = embeddings["word_embeddings"]

  # Activations at each layer.
  activations = module(dict(text=text),signature="activations", as_dict=True)
  activations = activations["activations"]

  # Negative log likelihood of the text, and perplexity.
  neg_log_likelihood = module(dict(text=text), signature="neg_log_likelihood",
                              as_dict=True)
  neg_log_likelihood = neg_log_likelihood["neg_log_likelihood"]
  ppl = tf.exp(tf.reduce_mean(neg_log_likelihood, axis=1))

  # Tokenization and detokenization with the sentencepiece model.
  token_ids = module(dict(text=text), signature="tokenization", as_dict=True)
  token_ids = token_ids["token_ids"]

  detoken_text = module(dict(token_ids=token_ids), signature="detokenization",
                        as_dict=True)
  detoken_text = detoken_text["text"]



  init_op = tf.group([tf.global_variables_initializer(),
                      tf.tables_initializer()])

# Initialize session.
with tf.Session(graph=g) as session:
  session.run(init_op)
  embeddings, neg_log_likelihood, ppl, activations, token_ids, detoken_text = session.run([
    embeddings, neg_log_likelihood, ppl, activations, token_ids, detoken_text])

In [ ]:
token_ids

array([[   13,     3,    13,    32,     7,  1060,    12,  6708,   198,
         4888,  6824,   577,  8469,    13,  1824,    65,   125, 12974,
           13,     4,    13,    54,    13,     5,    13,    33,  3322,
         9505, 20236,    35,     7, 16745,   219,  3174,  6761, 11421,
           19,     8, 10110,  3549,     7,    53,  1202,    32,     7,
         1060,    20,   546,    84,  1967,   315,    15,  4871, 13775,
         2624,    19,  2702,    27,   296,   736, 14103,   175,     9,
         2819,    10,   542]], dtype=int32)

In [ ]:
detoken_text[0].decode()

"_START_ARTICLE_ しのぶ・まさみshow'05 恋してラララ _START_SECTION_ 概要 _START_PARAGRAPH_ 『上海ルーキーSHOW』の打ち切り後に放送された年末特番で、同番組MCの大竹しのぶと久本雅美が恋愛にまつわるテーマでトークや音楽企画を展開していた。基本は女"

In [9]:
def create_token_explorer():
  num_tokens = 10000
  token_explorer = list(range(num_tokens))
  for i in range(num_tokens - 1):
    token_explorer.insert(num_tokens - 1 - i, 0)
  return np.array([token_explorer])

token_explorer = create_token_explorer()

In [7]:
#@title Original hub code
!pip install --quiet tensorflow-text
import numpy as np
import tensorflow.compat.v1 as tf
import tensorflow_hub as hub
import tensorflow_text as tf_text
tf.disable_eager_execution()

g = tf.Graph()
with g.as_default():
  module = hub.Module("https://tfhub.dev/google/wiki40b-lm-ja/1")
  detoken_text = module(dict(token_ids=token_explorer), signature="detokenization",
                        as_dict=True)
  detoken_text = detoken_text["text"]

  init_op = tf.group([tf.global_variables_initializer(),
                      tf.tables_initializer()])

# Initialize session.
with tf.Session(graph=g) as session:
  session.run(init_op)
  detoken_text = session.run([
    detoken_text])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [5]:
!pip install --quiet tensorflow-text
import numpy as np
import tensorflow.compat.v1 as tf
import tensorflow_hub as hub
import tensorflow_text as tf_text

def create_token_explorer():
  num_tokens = 32000
  token_explorer = list(range(num_tokens))
  for i in range(num_tokens - 1):
    token_explorer.insert(num_tokens - 1 - i, 4)
  return np.array([token_explorer])

token_explorer = create_token_explorer()

tf.disable_eager_execution()
g = tf.Graph()
with g.as_default():
  module = hub.Module("https://tfhub.dev/google/wiki40b-lm-ja/1")
  detoken_text = module(dict(token_ids=token_explorer), signature="detokenization",
                        as_dict=True)
  detoken_text = detoken_text["text"]

  init_op = tf.group([tf.global_variables_initializer(),
                      tf.tables_initializer()])

# Initialize session.
with tf.Session(graph=g) as session:
  session.run(init_op)
  detoken_text = session.run([
    detoken_text])
  
tokens = detoken_text[0][0].decode().split('_START_SECTION_')[8:]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [10]:
detoken_text[0][0].decode()

' ⁇ _START_SECTION__START_SECTION__START_SECTION__START_ARTICLE__START_SECTION__START_SECTION__START_SECTION__START_PARAGRAPH__START_SECTION__NEWLINE__START_SECTION_の_START_SECTION_、_START_SECTION_。_START_SECTION_は_START_SECTION_年_START_SECTION_・_START_SECTION_ _START_SECTION_)_START_SECTION_が_START_SECTION_(_START_SECTION_に_START_SECTION_を_START_SECTION_で_START_SECTION_と_START_SECTION_月_START_SECTION_」_START_SECTION_「_START_SECTION_2_START_SECTION_1_START_SECTION_から_START_SECTION_や_START_SECTION_3_START_SECTION_日_START_SECTION_である_START_SECTION_した_START_SECTION_し_START_SECTION_『_START_SECTION_も_START_SECTION_』_START_SECTION_として_START_SECTION_4_START_SECTION_する_START_SECTION_年に_START_SECTION_では_START_SECTION_5_START_SECTION_た_START_SECTION_6_START_SECTION_には_START_SECTION_また_START_SECTION_7_START_SECTION_10_START_SECTION_など_START_SECTION_第_START_SECTION_8_START_SECTION_9_START_SECTION_この_START_SECTION_大_START_SECTION_概要_START_SECTION_る_START_SECTION_12_START_SECTION_という_START_SECTION_ス

In [4]:
tokens = detoken_text[0][0].decode().split('_START_SECTION_')[8:]
tokens

['の',
 '、',
 '。',
 'は',
 '年',
 '・',
 ' ',
 ')',
 'が',
 '(',
 'に',
 'を',
 'で',
 'と',
 '月',
 '」',
 '「',
 '2',
 '1',
 'から',
 'や',
 '3',
 '日',
 'である',
 'した',
 'し',
 '『',
 'も',
 '』',
 'として',
 '4',
 'する',
 '年に',
 'では',
 '5',
 'た',
 '6',
 'には',
 'また',
 '7',
 '10',
 'など',
 '第',
 '8',
 '9',
 'この',
 '大',
 '概要',
 'る',
 '12',
 'という',
 'ス',
 ' (',
 'された',
 'その',
 'て',
 '-',
 '日に',
 'して',
 'している',
 '11',
 '人',
 'となった',
 'な',
 '市',
 ' -',
 'ている',
 '日本',
 ':',
 '中',
 '山',
 '町',
 '一',
 'により',
 '回',
 'であった',
 'による',
 '本',
 '昭和',
 '.',
 'され',
 'その後',
 'となる',
 'によって',
 '後',
 'ア',
 'であり',
 '子',
 ',',
 '月に',
 '15',
 '長',
 'より',
 'ト',
 '17',
 '上',
 '川',
 '新',
 'か',
 '部',
 'がある',
 '同',
 'ズ',
 'り',
 '20',
 '元',
 'ため',
 'ク',
 '16',
 'へ',
 'ド',
 '名',
 '同年',
 'まで',
 'm',
 '年には',
 '駅',
 '家',
 'ラ',
 '、「',
 '郡',
 'しかし',
 '国',
 '線',
 'カ',
 'される',
 '東',
 'A',
 '高',
 's',
 '道',
 'となり',
 '18',
 '号',
 'ナ',
 'い',
 '性',
 '三',
 '大学',
 'などの',
 '14',
 '村',
 '南',
 '年から',
 '30',
 'B',
 'だった',
 '法',
 'への',
 'において',
 '西',
 '下',
 

# Training our own tokenisation with google SentencePiece

---



By reading the paper for Wiki-40b (https://storage.googleapis.com/pub-tools-public-publication-data/pdf/18cd66cc7d31ce4c724cef1d2755b417f74de27c.pdf), it is clear that they do not use anything extra except the statistics based SentencePiece for tokenisation.

https://github.com/google/sentencepiece

---


https://github.com/google/sentencepiece/tree/master/python


---



https://github.com/google/sentencepiece/blob/master/python/sentencepiece_python_module_example.ipynb

---

All options for training can be viewed here: https://github.com/google/sentencepiece/blob/master/doc/options.md Note this was found by reading the readme on the main page


In [6]:
#@title Install
!pip install sentencepiece
!wget https://raw.githubusercontent.com/google/sentencepiece/master/data/botchan.txt

import sentencepiece as spm

--2021-02-22 03:48:57--  https://raw.githubusercontent.com/google/sentencepiece/master/data/botchan.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 278779 (272K) [text/plain]
Saving to: ‘botchan.txt.1’

botchan.txt.1       100%[===================>] 272.25K  --.-KB/s    in 0.04s   

2021-02-22 03:48:57 (6.91 MB/s) - ‘botchan.txt.1’ saved [278779/278779]



In [ ]:
#@title Download the test folder from github
!mkdir test
!wget https://raw.githubusercontent.com/google/sentencepiece/master/data/botchan.txt -O /content/test/botchan.txt
!wget https://raw.githubusercontent.com/google/sentencepiece/master/python/test/test_model.model -O /content/test/test_model.model


mkdir: cannot create directory ‘test’: File exists
--2021-02-11 02:28:12--  https://raw.githubusercontent.com/google/sentencepiece/master/data/botchan.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 278779 (272K) [text/plain]
Saving to: ‘/content/test/botchan.txt’

/content/test/botch 100%[===================>] 272.25K  --.-KB/s    in 0.02s   

2021-02-11 02:28:12 (16.6 MB/s) - ‘/content/test/botchan.txt’ saved [278779/278779]

--2021-02-11 02:28:12--  https://raw.githubusercontent.com/google/sentencepiece/master/python/test/test_model.model
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... co

## Tokenisation exploration of pretrained example

In [5]:
import sentencepiece as spm
sp = spm.SentencePieceProcessor(model_file='test/test_model.model')

OSError: ignored

In [ ]:
sp.encode('This is a test')

[284, 47, 11, 4, 15, 400]

In [ ]:
sp.encode(['This is a test', 'Hello world'], out_type=int)

[[284, 47, 11, 4, 15, 400], [151, 88, 21, 887]]

In [ ]:
sp.encode('This is a test', out_type=str)

['▁This', '▁is', '▁a', '▁', 't', 'est']

In [ ]:
sp.encode(['This is a test', 'Hello world'], out_type=str)

[['▁This', '▁is', '▁a', '▁', 't', 'est'], ['▁He', 'll', 'o', '▁world']]

In [ ]:
for _ in range(10):
  print(sp.encode('This is a test', out_type=str, enable_sampling=True, alpha=0.1, nbest_size=-1))

['▁T', 'h', 'is', '▁is', '▁a', '▁', 't', 'est']
['▁', 'This', '▁is', '▁a', '▁', 'te', 's', 't']
['▁T', 'h', 'i', 's', '▁is', '▁', 'a', '▁', 'te', 'st']
['▁', 'T', 'h', 'i', 's', '▁', 'i', 's', '▁a', '▁', 't', 'e', 'st']
['▁', 'This', '▁', 'i', 's', '▁', 'a', '▁', 'te', 's', 't']
['▁This', '▁is', '▁', 'a', '▁', 'te', 's', 't']
['▁This', '▁', 'i', 's', '▁', 'a', '▁', 't', 'e', 's', 't']
['▁T', 'h', 'i', 's', '▁', 'is', '▁a', '▁', 'te', 's', 't']
['▁', 'T', 'h', 'is', '▁', 'i', 's', '▁a', '▁', 'te', 'st']
['▁This', '▁', 'is', '▁a', '▁', 'te', 's', 't']


In [ ]:
sp.decode([284, 47, 11, 4, 15, 400])

'This is a test'

In [ ]:
sp.decode([[284, 47, 11, 4, 15, 400], [151, 88, 21, 887]])

['This is a test', 'Hello world']

In [ ]:
sp.decode(['▁', 'This', '▁', 'is', '▁a', '▁', 't', 'e', 'st'])

'This is a test'

In [ ]:
sp.decode([['▁This', '▁is', '▁a', '▁', 't', 'est'], ['▁He', 'll', 'o', '▁world']])

['This is a test', 'Hello world']

In [ ]:
sp.get_piece_size()

1000

In [ ]:
sp.id_to_piece(2)


'</s>'

In [ ]:
sp.id_to_piece([2, 3, 4])

['</s>', '\r', '▁']

In [ ]:
sp.piece_to_id('<s>')

1

In [ ]:
sp.piece_to_id(['</s>', '\r', '▁'])

[2, 3, 4]

In [ ]:
len(sp)

1000

In [ ]:
sp['</s>']

2

## Training example

In [ ]:
import sentencepiece as spm
#spm.SentencePieceTrainer.train(input='test/botchan.txt', model_prefix='m', vocab_size=1000, user_defined_symbols=['foo', 'bar'], hard_vocab_limit = False)
#spm.SentencePieceTrainer.train(input='test/botchan.txt', model_prefix='m', vocab_size=1000, user_defined_symbols=['foo', 'bar'])
spm.SentencePieceTrainer.train(input='test/botchan.txt', model_prefix='m', vocab_size=1000)


In [ ]:
sp = spm.SentencePieceProcessor(model_file='m.model')

In [ ]:
sp.vocab_size()

1000

In [ ]:
len(sp)

1000

In [ ]:
sp.id_to_piece([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])

['<unk>',
 '<s>',
 '</s>',
 ',',
 '▁',
 '.',
 '▁the',
 's',
 '▁I',
 '▁to',
 '▁a',
 'ed',
 'e',
 't',
 '▁and',
 '▁of',
 'ing',
 'a']

In [ ]:
spm.SentencePieceTrainer.train(input='test/botchan.txt', model_prefix='m', vocab_size=1000, user_defined_symbols=[' ', 'bar'])

RuntimeError: ignored

## SPM From Japanese Bible

In [30]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [2]:
import os
files = os.listdir('/gdrive/MyDrive/Japanese/Bible/chapters')
files = ['/gdrive/MyDrive/Japanese/Bible/chapters/' + i for i in files if '.txt' in i]

In [7]:
spm.SentencePieceTrainer.train(input=files, model_prefix='m', vocab_size=32000)

In [8]:
sp = spm.SentencePieceProcessor(model_file='m.model')

In [22]:
import numpy as np

def create_token_explorer():
  num_tokens = 32000
  token_explorer = list(range(num_tokens))
  for i in range(num_tokens - 1):
    token_explorer.insert(num_tokens - 1 - i, 4)
  return np.array([token_explorer])

token_explorer = create_token_explorer().tolist()

In [25]:
tokens = sp.id_to_piece(list(range(32000)))

In [27]:
import pickle

the_list = pickle.load(open("/gdrive/MyDrive/Japanese/Bible/chapters/kanji_list", "rb"))

In [33]:
len(the_list_names)


13305

In [91]:
the_list_names = [entry['name'] for entry in the_list]
not_present = []

for i, character in enumerate(the_list_names):
  inside = False
  for word in tokens:
    if character in word:
      inside = True
      break
  if not inside:
    not_present.append((character, i))

    #print("The character ", character, " is not representable. It has index ", i)

In [29]:
#for entry in not_present:
  #print("The character ", entry[0], " is not representable. It has index ", entry[1])
print(len(the_list_names))
print(len(not_present))


13305
11576


In [85]:
the_list_names_space_dropped = the_list_names.copy()
the_list_names_space_dropped.remove(' ')

In [94]:
#spm.SentencePieceTrainer.train(input=files, model_prefix='few_lines', vocab_size=32000, user_defined_symbols=the_list_names)
spm.SentencePieceTrainer.train(input=files, model_prefix='letters', vocab_size=13310, user_defined_symbols=the_list_names_space_dropped)

In [95]:
sp = spm.SentencePieceProcessor(model_file='letters.model')

In [97]:
!cp letters.model /gdrive/MyDrive/Japanese/Bible/
!cp letters.vocab /gdrive/MyDrive/Japanese/Bible/

In [ ]:
pickle.dump(kanji_list, open("/gdrive/MyDrive/Japanese/Bible/chapters/kanji_list", "wb+"))

In [96]:
tokens = sp.id_to_piece(list(range(13310)))

In [92]:
#for entry in not_present:
  #print("The character ", entry[0], " is not representable. It has index ", entry[1])
print(len(the_list_names))
print(len(not_present))

13305
1


In [93]:
not_present

[(' ', 14)]

In [53]:
tokens

['<unk>',
 '<s>',
 '</s>',
 '、',
 '。',
 '▁',
 'は',
 'の',
 'ました',
 'と',
 'が',
 'に',
 'を',
 '▁「',
 '。」',
 'も',
 'で',
 'ます',
 'る',
 'から',
 'て',
 '」',
 'なさい',
 'た',
 'です',
 '▁しかし',
 'か',
 'や',
 'その',
 '・',
 'のです',
 '、「',
 '▁8',
 '▁5',
 '』',
 '▁10',
 '▁7',
 '▁9',
 '▁3',
 '▁4',
 'だ',
 '▁6',
 '▁2',
 'り',
 '▁11',
 'し',
 '▁12',
 '▁13',
 'す',
 ')',
 'い',
 '▁14',
 '▁15',
 '▁16',
 'よ',
 'この',
 '▁17',
 '(',
 '▁18',
 '▁19',
 'した',
 'からです',
 'には',
 '▁彼らは',
 'でした',
 '▁20',
 '▁21',
 'ない',
 '▁22',
 '▁また',
 '▁そして',
 'な',
 '私は',
 '人',
 'のように',
 'また',
 '▁23',
 '▁その',
 'する',
 'まで',
 '▁この',
 'く',
 'ように',
 '▁24',
 '▁私は',
 '言いました',
 'たのです',
 'って',
 'である',
 'なら',
 'わたしは',
 '▁25',
 '▁『',
 '▁26',
 '、『',
 '▁ところが',
 '▁主は',
 '主は',
 '神',
 'でしょう',
 '▁こうして',
 'のだ',
 '私',
 'った',
 'して',
 '▁27',
 '▁わたしは',
 '神の',
 'よう',
 '主',
 '▁そこで',
 '▁28',
 'ている',
 '者',
 'あなた',
 'お',
 'という',
 '王',
 '▁それから',
 'ことを',
 'しました',
 'ら',
 'として',
 '▁29',
 '▁それは',
 'ると',
 '▁神は',
 '▁その時',
 'でも',
 '彼らは',
 '▁すると',
 '私の',
 'など',
 'あなたの',
 'き',
 'ません'

In [47]:
spm.SentencePieceTrainer.train(input=files, model_prefix='few_lines', vocab_size=32000, seed_sentencepiece_size=1)

# Training our own tokenisation with tensorflow text encoders

https://github.com/tensorflow/text/blob/master/docs/api_docs/python/text.md

https://blog.tensorflow.org/2019/06/introducing-tftext.html

# Training an english model on Librispeech subset

In [ ]:
import tensorflow_datasets as tfds

In [ ]:
ds = tfds.load('mnist', split='train', shuffle_files=True)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.





Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [ ]:
tfds.list_builders()

['abstract_reasoning',
 'accentdb',
 'aeslc',
 'aflw2k3d',
 'ag_news_subset',
 'ai2_arc',
 'ai2_arc_with_ir',
 'amazon_us_reviews',
 'anli',
 'arc',
 'bair_robot_pushing_small',
 'bccd',
 'beans',
 'big_patent',
 'bigearthnet',
 'billsum',
 'binarized_mnist',
 'binary_alpha_digits',
 'blimp',
 'bool_q',
 'c4',
 'caltech101',
 'caltech_birds2010',
 'caltech_birds2011',
 'cars196',
 'cassava',
 'cats_vs_dogs',
 'celeb_a',
 'celeb_a_hq',
 'cfq',
 'chexpert',
 'cifar10',
 'cifar100',
 'cifar10_1',
 'cifar10_corrupted',
 'citrus_leaves',
 'cityscapes',
 'civil_comments',
 'clevr',
 'clic',
 'clinc_oos',
 'cmaterdb',
 'cnn_dailymail',
 'coco',
 'coco_captions',
 'coil100',
 'colorectal_histology',
 'colorectal_histology_large',
 'common_voice',
 'coqa',
 'cos_e',
 'cosmos_qa',
 'covid19sum',
 'crema_d',
 'curated_breast_imaging_ddsm',
 'cycle_gan',
 'deep_weeds',
 'definite_pronoun_resolution',
 'dementiabank',
 'diabetic_retinopathy_detection',
 'div2k',
 'dmlab',
 'downsampled_imagenet',
 

In [ ]:
# >>> ds2 = tfds.load('librispeech', split='train_clean100', shuffle_files=True)
# Not enough disk space. Needed: 361.62 GiB (download: 57.14 GiB, generated: 304.47 GiB)

In [ ]:
tfds.is_dataset_on_gcs('librispeech')

False

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
with open('/gdrive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat '/gdrive/My Drive/foo.txt'

In [ ]:
ds2 = tfds.load('librispeech', split='train_clean100', shuffle_files=True, data_dir='/gdrive/My Drive/datasets/librispeech')

OSError: ignored

In [ ]:
ds2 = tfds.load('mnist', split='train', shuffle_files=True, data_dir='/gdrive/My Drive/datasets/mnist')

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.





Dataset mnist downloaded and prepared to /gdrive/My Drive/datasets/mnist/mnist/3.0.1. Subsequent calls will reuse this data.


In [ ]:
import shutil
import os

In [ ]:
shutil.disk_usage(os.path.abspath('/gdrive/My Drive/datasets/mnist')).free

77183885312

In [ ]:
_URL = "http://www.openslr.org/12"
_DL_URL = "http://www.openslr.org/resources/12/"
_DL_URLS = {
    "dev_clean": _DL_URL + "dev-clean.tar.gz",
    "dev_other": _DL_URL + "dev-other.tar.gz",
    "test_clean": _DL_URL + "test-clean.tar.gz",
    "test_other": _DL_URL + "test-other.tar.gz",
    "train_clean100": _DL_URL + "train-clean-100.tar.gz",
    "train_clean360": _DL_URL + "train-clean-360.tar.gz",
    "train_other500": _DL_URL + "train-other-500.tar.gz",
}

In [ ]:
train100 = _DL_URLS['train_clean100']

In [ ]:
!wget $train100 -O /gdrive/My\ Drive/datasets/train100.tar.gz

--2021-02-11 05:50:25--  http://www.openslr.org/resources/12/train-clean-100.tar.gz
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6387309499 (5.9G) [application/x-gzip]
Saving to: ‘/gdrive/My Drive/datasets/train100.tar.gz’

/gdrive/My Drive/da 100%[===================>]   5.95G  25.9MB/s    in 4m 2s   

2021-02-11 05:54:27 (25.2 MB/s) - ‘/gdrive/My Drive/datasets/train100.tar.gz’ saved [6387309499/6387309499]



In [ ]:
train100

'http://www.openslr.org/resources/12/train-clean-100.tar.gz'

In [ ]:
!tar -xvf /gdrive/My\ Drive/datasets/train100.tar.gz -C /gdrive/My\ Drive/datasets/librispeech

Streaming output truncated to the last 5000 lines.
LibriSpeech/train-clean-100/2136/5143/2136-5143-0024.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0012.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0044.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0003.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0010.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0025.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0043.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0004.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0026.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0015.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0032.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0027.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0039.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0020.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0023.flac
LibriSpeech/train-clean-100/2136/5143/2136-5143-0007.flac
LibriSpeech/train-cle

In [ ]:
ds2 = tfds.load('librispeech', split='train_clean100', shuffle_files=True, data_dir='/gdrive/My Drive/datasets/')

Not enough disk space. Needed: 361.62 GiB (download: 57.14 GiB, generated: 304.47 GiB)


ConnectionError: ignored

In [ ]:
!pip install git+https://github.com/Jaidon-Smith/datasets.git

  Cloning https://github.com/Jaidon-Smith/datasets.git to /tmp/pip-req-build-27_qqnwv
  Running command git clone -q https://github.com/Jaidon-Smith/datasets.git /tmp/pip-req-build-27_qqnwv
  Created wheel for tensorflow-datasets: filename=tensorflow_datasets-4.2.0+nightly-cp36-none-any.whl size=3791426 sha256=62deaaddc341202949542183adde9f13d0244888a6fdd980b17453a311e6bed7
  Stored in directory: /tmp/pip-ephem-wheel-cache-z0vnpxq4/wheels/11/7a/47/6fb04ed624933e14863b94c2ac1f123844a1c0805ed845dc97
Successfully built tensorflow-datasets
  Found existing installation: tensorflow-datasets 4.0.1
    Uninstalling tensorflow-datasets-4.0.1:
      Successfully uninstalled tensorflow-datasets-4.0.1


In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 7555789263929026378, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14638920512
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 1262428234004142416
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]